Dummy Content Datase


In [1]:
import pandas as pd

# Dummy content dataset
content_data = [
    [1, "C1", 5, "entertainment", "comedy", "high", 0.2],
    [2, "C2", 2, "annoying", "prank", "high", 0.9],
    [3, "C3", 4, "educational", "tech", "low", 0.3],
    [4, "C1", 5, "entertainment", "motivation", "medium", 0.4],
    [5, "C4", 1, "time_pass", "trend", "high", 0.95],
    [6, "C5", 3, "entertainment", "music", "medium", 0.6],
    [7, "C6", 4, "educational", "science", "low", 0.2],
    [8, "C2", 2, "annoying", "challenge", "high", 0.85],
    [9, "C7", 5, "entertainment", "cinematic", "medium", 0.1],
    [10, "C8", 3, "time_pass", "vlog", "medium", 0.5]
]

columns = [
    "content_id",
    "creator_id",
    "creator_rating",
    "category",
    "genre",
    "energy_level",
    "repetition_score"
]

content_df = pd.DataFrame(content_data, columns=columns)

content_df


,content_id,creator_id,creator_rating,category,genre,energy_level,repetition_score
0,1,C1,5,entertainment,comedy,high,0.20
1,2,C2,2,annoying,prank,high,0.90
2,3,C3,4,educational,tech,low,0.30
3,4,C1,5,entertainment,motivation,medium,0.40
4,5,C4,1,time_pass,trend,high,0.95
5,6,C5,3,entertainment,music,medium,0.60
6,7,C6,4,educational,science,low,0.20
7,8,C2,2,annoying,challenge,high,0.85
8,9,C7,5,entertainment,cinematic,medium,0.10
9,10,C8,3,time_pass,vlog,medium,0.50


User Behavior Input

In [2]:
# Simulated user behavior input
user_behavior = {
    "scroll_speed": "fast",        # fast / medium / slow
    "avg_watch_time": 3,           # seconds
    "rewatch_count": 0,            # number of rewatches
    "skip_rate": 0.75,             # 0 to 1
    "session_length": 18,          # minutes
    "recent_category_bias": "entertainment"
}

user_behavior


{'scroll_speed': 'fast',
 'avg_watch_time': 3,
 'rewatch_count': 0,
 'skip_rate': 0.75,
 'session_length': 18,
 'recent_category_bias': 'entertainment'}

Emotion Detection from User Behavior

In [3]:
def detect_emotion(user_behavior):
    if user_behavior["rewatch_count"] > 1:
        return "attached"

    if user_behavior["scroll_speed"] == "fast" and user_behavior["skip_rate"] > 0.6:
        return "bored"

    if user_behavior["scroll_speed"] == "slow" and user_behavior["avg_watch_time"] > 6:
        return "engaged"

    if user_behavior["session_length"] > 20 and user_behavior["scroll_speed"] == "fast":
        return "overstimulated"

    return "neutral"


user_emotion = detect_emotion(user_behavior)
user_emotion


'bored'

Anti-Repetition Filter

In [4]:
def apply_anti_repetition(content_df, user_emotion):
    filtered = content_df.copy()

    if user_emotion == "bored":
        filtered = filtered[
            (filtered["repetition_score"] < 0.7) &
            (filtered["category"] != "annoying")
        ]

    return filtered


Exploration–Exploitation Balance


In [5]:
import random

def exploration_exploitation(content_df, user_behavior, user_emotion):
    main_interest = user_behavior["recent_category_bias"]

    if user_emotion in ["engaged", "attached"]:
        return content_df[content_df["category"] == main_interest]

    else:
        # exploration: mix categories
        return content_df.sample(min(5, len(content_df)))


Creator Quality Ranking

In [6]:
def rank_content(content_df):
    return content_df.sort_values(
        by=["creator_rating", "repetition_score"],
        ascending=[False, True]
    )


Full Pipeline

In [7]:
def recommend_content(content_df, user_behavior):
    emotion = detect_emotion(user_behavior)

    filtered = apply_anti_repetition(content_df, emotion)

    balanced = exploration_exploitation(filtered, user_behavior, emotion)

    ranked = rank_content(balanced)

    return emotion, ranked.head(5)


Run the System

In [8]:
emotion, recommendations = recommend_content(content_df, user_behavior)

print("Detected Emotion:", emotion)
recommendations


Detected Emotion: bored


,content_id,creator_id,creator_rating,category,genre,energy_level,repetition_score
0,1,C1,5,entertainment,comedy,high,0.2
6,7,C6,4,educational,science,low,0.2
2,3,C3,4,educational,tech,low,0.3
9,10,C8,3,time_pass,vlog,medium,0.5
5,6,C5,3,entertainment,music,medium,0.6


Explanation Generator

In [9]:
def explain_recommendation(row, user_emotion, user_behavior):
    reasons = []

    reasons.append(f"User emotion detected as '{user_emotion}'.")

    if row["creator_rating"] >= 4:
        reasons.append("High-quality creator.")

    if row["repetition_score"] < 0.4:
        reasons.append("Low repetition content.")

    if row["category"] == user_behavior["recent_category_bias"]:
        reasons.append("Matches your main interest.")
    else:
        reasons.append("Introduced for exploration and freshness.")

    return " ".join(reasons)


Attach Explanations to Recommendations

In [10]:
def add_explanations(recommendations, user_emotion, user_behavior):
    explanations = []

    for _, row in recommendations.iterrows():
        explanations.append({
            "content_id": row["content_id"],
            "creator_id": row["creator_id"],
            "category": row["category"],
            "explanation": explain_recommendation(row, user_emotion, user_behavior)
        })

    return explanations


Run with Explanations

In [11]:
emotion, recommendations = recommend_content(content_df, user_behavior)
final_output = add_explanations(recommendations, emotion, user_behavior)

for item in final_output:
    print(item)


{'content_id': 1, 'creator_id': 'C1', 'category': 'entertainment', 'explanation': "User emotion detected as 'bored'. High-quality creator. Low repetition content. Matches your main interest."}
{'content_id': 4, 'creator_id': 'C1', 'category': 'entertainment', 'explanation': "User emotion detected as 'bored'. High-quality creator. Matches your main interest."}
{'content_id': 7, 'creator_id': 'C6', 'category': 'educational', 'explanation': "User emotion detected as 'bored'. High-quality creator. Low repetition content. Introduced for exploration and freshness."}
{'content_id': 3, 'creator_id': 'C3', 'category': 'educational', 'explanation': "User emotion detected as 'bored'. High-quality creator. Low repetition content. Introduced for exploration and freshness."}
{'content_id': 6, 'creator_id': 'C5', 'category': 'entertainment', 'explanation': "User emotion detected as 'bored'. Matches your main interest."}


Streamlit Skeleton

In [12]:

print("\n--- DEMO SCENARIO: ENGAGED USER ---")

user_behavior["scroll_speed"] = "slow"
user_behavior["avg_watch_time"] = 8
user_behavior["skip_rate"] = 0.2

emotion, recommendations = recommend_content(content_df, user_behavior)

print("DETECTED EMOTION:", emotion)
display(recommendations)




--- DEMO SCENARIO: ENGAGED USER ---
DETECTED EMOTION: engaged


,content_id,creator_id,creator_rating,category,genre,energy_level,repetition_score
8,9,C7,5,entertainment,cinematic,medium,0.1
0,1,C1,5,entertainment,comedy,high,0.2
3,4,C1,5,entertainment,motivation,medium,0.4
5,6,C5,3,entertainment,music,medium,0.6
